2.1.1 Age and Gender

https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/664944/Young-people-statistics-data-tables-from-the-national-drug-treatment-monitoring-system-2016-2017.xls

In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/664944/'\
                    'Young-people-statistics-data-tables-from-the-national-drug-treatment-monitoring-system-2016-2017.xls'
    inputFile = sourceFolder / 'Young-people-statistics-data-tables-from-the-national-drug-treatment-monitoring-system-2016-2017.xls'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [2]:
tab = loadxlstabs(inputFile, sheetids='2.1.1 Age and Gender')[0]

Loading in\Young-people-statistics-data-tables-from-the-national-drug-treatment-monitoring-system-2016-2017.xls which has size 281600 bytes
Table names: ['2.1.1 Age and Gender']


In [3]:
observations = tab.excel_ref('B5').expand(DOWN).expand(RIGHT).is_not_blank()
observations

{<C9 0.27>, <C5 0.0>, <D6 179.0>, <C8 0.19>, <E11 0.26>, <C7 0.07>, <B6 105.0>, <F12 16436.0>, <C12 1.0>, <B9 1555.0>, <B7 396.0>, <B10 1315.0>, <F6 284.0>, <E10 0.26>, <B8 1061.0>, <E9 0.26>, <G5 0.0>, <B12 5669.0>, <C10 0.23>, <D9 2791.0>, <G7 0.06>, <D7 594.0>, <D8 1549.0>, <E6 0.02>, <G10 0.25>, <D5 56.0>, <C6 0.02>, <F11 4029.0>, <D12 10767.0>, <G9 0.26>, <B5 12.0>, <F9 4346.0>, <E8 0.14>, <E12 1.0>, <E5 0.01>, <F8 2610.0>, <F5 68.0>, <D11 2804.0>, <C11 0.22>, <F10 4109.0>, <G12 1.0>, <B11 1225.0>, <F7 990.0>, <G11 0.25>, <E7 0.06>, <G6 0.02>, <G8 0.16>, <D10 2794.0>}

In [5]:
age = tab.excel_ref('A5').expand(DOWN).is_not_blank()
age

{<A11 '17-18'>, <A9 '15-16'>, <A5 'Under 12'>, <A6 '12-13'>, <A12 'Total clients'>, <A7 '13-14'>, <A8 '14-15'>, <A10 '16-17'>}

In [6]:
sex = tab.excel_ref('B3').expand(RIGHT).is_not_blank() 
sex

{<D3 'Male'>, <B3 'Female'>, <F3 'Persons'>}

In [7]:
measuretype = tab.excel_ref('B4').expand(RIGHT).is_not_blank() 
measuretype

{<B4 'n'>, <G4 '%'>, <D4 'n'>, <E4 '%'>, <C4 '%'>, <F4 'n'>}

In [ ]:
Dimensions = [
            HDim(clients,'Clients in treatment',CLOSEST,ABOVE),
            HDimConst('Measure Type','Count'),
            HDim(treatmentstatus, 'Treatment Status',DIRECTLY,LEFT),
            HDim(period, 'Period',DIRECTLY,ABOVE),
            HDimConst('Unit','People')            
            ]

In [ ]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# if is_interactive():
#     savepreviewhtml(c1)

In [ ]:
new_table = c1.topandas()
new_table

In [ ]:
new_table = new_table[new_table['OBS'] != 0 ]

In [ ]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [ ]:
new_table.head()

In [ ]:
new_table['Measure Type'].unique()

In [ ]:
new_table.tail()

In [ ]:
new_table.dtypes

In [ ]:
new_table['Value'] = new_table['Value'].astype(str)

In [ ]:
new_table.head(3)

In [ ]:
def user_perc(x,y):
    
    if str(x) == '%':
        return 'Percentage'
    else:
        return y
    
new_table['Measure Type'] = new_table.apply(lambda row: user_perc(row['Period'],row['Measure Type']), axis = 1)


In [ ]:
new_table['Period'] = new_table['Period'].map(
    lambda x: {
        'Total' : 'All years',
        '%'     : 'All years'
        }.get(x, x))

In [ ]:
new_table.tail()

In [ ]:
new_table = new_table[['Period','Treatment Status','Clients in treatment','Measure Type','Value','Unit']]

In [ ]:
if is_interactive():
    SubstancetinationFolder = Path('out')
    SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
    new_table.to_csv(SubstancetinationFolder / ('Appendix B.csv'), index = False)